# Learn Lyncs-API

In [24]:
import os
import sys
import numpy as np  # type: ignore
import lyncs_io as lio  # type: ignore
from typing import List

<div class="alert alert-block alert-warning"><b>Warning:</b>If you get the warning <code><FONT COLOR="#db5048">ModuleNotFoundError</code><code>: No module named 'lyncs_io'</code>, you need to change the kernel via Kernel > Change kernel > lyncs.</div>

In [25]:
import plaquette as RPlaq

## Introduction

<a href="https://github.com/Lyncs-API">Lyncs-API</a> is a Python API for Lattice QCD applications () created by Simone Bacchio, Christodoulos Stylianou and Alexandros Angeli.

One such API is <a href="https://github.com/Lyncs-API/lyncs.io">lyncs.io</a>, a suite of I/O functions which allow for quick and simple interfacing with several common file formats, including lattice-oriented formats such as `.lime` and `.openqcd`.

## Accessing the data

Let us use some example gaugefield files to showcase `lyncs`. We use $8\times24^3$ SU(3) $N_f = 2 + 1$ gaugefields in $3+1d$. These otherwise have the same parameters as FASTSUM's Generation 2 ensembles \[[1](https://arxiv.org/abs/1412.6411), [2](https://arxiv.org/abs/2007.04188)\].

In [26]:
# the location, name and ID number of the gaugefields
gfDir = 'confs'
gfName = 'Gen2_8x24n'
gfIDs = [7,8,9]

Using the `head()` function, we can read the header data stored in the file and access the information in the form of a `dict` object.

Here we see that our example gaugefield files are arranged in the shape $N_t \times N_s^3 \times N_d \times N_c^2$. The `dtype` key shows that the datatype is `'<c16'` or (little-endian) double-precision complex. `'_offset'` is the number of bytes in the header and `'plaq'` is the average value of the spatial and temporal plaquette.

In [27]:
# We can probe the header data of the gaugefield files
# Loop over each ID
for iid in gfIDs:
    gfFile = os.path.join(gfDir,f'{gfName}{iid}')
    # Read and print header
    print(f"{gfFile}:", lio.head(gfFile, format='openqcd'))

confs/Gen2_8x24n7: {'shape': (8, 24, 24, 24, 4, 3, 3), 'dtype': '<c16', '_offset': 24, 'plaq': 1.6265985010264397}
confs/Gen2_8x24n8: {'shape': (8, 24, 24, 24, 4, 3, 3), 'dtype': '<c16', '_offset': 24, 'plaq': 1.6235420123884416}
confs/Gen2_8x24n9: {'shape': (8, 24, 24, 24, 4, 3, 3), 'dtype': '<c16', '_offset': 24, 'plaq': 1.6244340856720185}


The `load()` function returns a `numpy.ndarray` containing the gaugefields. Let's load some example data:

In [28]:
# Load the gaugefields
# Make a list of gaugefield data
gfData: List[np.ndarray] = []
# Loop over each ID
for iid in gfIDs:
    gfFile = os.path.join(gfDir,f'{gfName}{iid}')
    # Load and append
    # Here we have specified the full path and the format
    # Can figure it out based on extension, but format is clearer
    gfData.append(lio.load(gfFile, format='openqcd'))
# Convert to array for better indexing
gfAr = np.asarray(gfData)

`lyncs.io` can also be used to convert from one format to another. For example, we can simply convert from `openqcd` format to `lime` using the `save()` function:

In [29]:
# Save the gaugefield array gfAr to a lime file
lio.save(gfAr,'Gen2_8x24_gfAr.lime')

Since our new file has a standard extension, `lyncs.io` can infer the format from the filename. The `head()` function now accesses the `lime` record associated with our new data.

In [30]:
# Read the header of our new lime file
lio.head('Gen2_8x24_gfAr.lime')

{'_lyncs_io': '0.2.3',
 'created': '2023-11-02 14:37:29',
 'type': "<class 'numpy.ndarray'>",
 'shape': (3, 8, 24, 24, 24, 4, 3, 3),
 'dtype': dtype('>c16'),
 'fortran_order': False,
 'descr': '<c16',
 'nbytes': 191102976,
 '_offset': 856}

If we want to access all of the records in the file, we can use `lime.read_records()`:

In [31]:
lio.lime.read_records('Gen2_8x24_gfAr.lime')

[{'_fp': <_io.BufferedReader name='Gen2_8x24_gfAr.lime'>,
  'magic_number': 1164413355,
  'version': 1,
  'msg_bits': 32768,
  'nbytes': 181,
  'lime_type': 'xlf-info',
  'offset': 144,
  'begin': True,
  'end': False,
  'data': b"_lyncs_io = 0.2.3\ncreated = 2023-11-02 14:37:29\ntype = <class 'numpy.ndarray'>\nshape = (3, 8, 24, 24, 24, 4, 3, 3)\ndtype = >c16\nfortran_order = False\ndescr = <c16\nnbytes = 191102976"},
 {'_fp': <_io.BufferedReader name='Gen2_8x24_gfAr.lime'>,
  'magic_number': 1164413355,
  'version': 1,
  'msg_bits': 0,
  'nbytes': 237,
  'lime_type': 'lyncs-io-info',
  'offset': 472,
  'begin': False,
  'end': False,
  'data': b"\x80\x04\x95\xe2\x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\t_lyncs_io\x94\x8c\x050.2.3\x94\x8c\x07created\x94\x8c\x132023-11-02 14:37:29\x94\x8c\x04type\x94\x8c\x17<class 'numpy.ndarray'>\x94\x8c\x05shape\x94(K\x03K\x08K\x18K\x18K\x18K\x04K\x03K\x03t\x94\x8c\x05dtype\x94\x8c\x05numpy\x94\x8c\x05dtype\x94\x93\x94\x8c\x03c16\x94\x89\x88\x87\x94R\x9

If we want to read this lime file back into a `numpy.ndarray` we can use the `load()` function again

In [32]:
lime = lio.load('Gen2_8x24_gfAr.lime')

## Manipulating the data

As an exercise, let's calculate the values for the whole, spatial and temporal plaquettes:
<img src="latticePlaqDiag.png" alt="plaquette diagram" width="400" height="400"/>

In [33]:
# Here show the shape of the data
# it is [ID, NT, NX, NY, NZ, mu, colour, colour]
print(gfAr.shape)

(3, 8, 24, 24, 24, 4, 3, 3)


In [34]:
# Use Ryan's plaquette code to calculate whole of lattice plaquette
# the sum of plaquettes, the number of plaquttes, the average plaquette and the time taken
sumTrP, nP, ave, time = RPlaq.plaquette(gfAr[0, ...])
print(f'calculated average plaquette {ave} in {time:.2} seconds')

calculated average plaquette 1.626598501026478 in 2.4e+01 seconds


In [35]:
# Use Ryan's plaquette code to calculate spatial plaquette
ssumTrP, snP, save, stime = RPlaq.plaquette(gfAr[0, ...], muStart=1, muEnd=4, nuEnd=4)
print(f'calculated average spatial plaquette {save} in {stime:.2} seconds')

calculated average spatial plaquette 1.0088805585220895 in 1.2e+01 seconds


In [36]:
# Use Ryan's plaquette code to calculate temporal plaquette
tsumTrP, tnP, tave, ttime = RPlaq.plaquette(gfAr[0, ...], muStart=0, muEnd=1, nuEnd=4)
print(f'calculated average temporal plaquette {tave} in {ttime:.2} seconds')

calculated average temporal plaquette 2.2443164435307277 in 1.2e+01 seconds


In [37]:
# Just check that this agrees with the whole of lattice plaquette (visually)
print(f'average of temporal and spatial plaquettes {(tave+save)/2.0}')

average of temporal and spatial plaquettes 1.6265985010264086


# C

Now that we can load and manipulate the data, we might want to increase performance by offloading our calculations to a script written in a compiled language such as C.

In [38]:
# Prepare a single gaugefield for writing
COrder = gfAr[0, ...]
# Now save
COrder.tofile('Gen2_8x24_gfAr0.C')

There is a C program `readC.c` supplied. This program will read `Gen2_8x24_gfAr0.C` in and calculate whole, spatial and temporal plaquettes. We do that now

In [39]:
# The command to compile
ccmd = 'gcc -O3 readC.c'
os.system(ccmd)

0

In [40]:
# The command to run
rcmd = './a.out'
os.system(rcmd)

Type:		sumReTrP:	nP:	Avg:
whole		1079332.688553	663552	1.626599
spatial		334722.356184	331776	1.008881
temporal	744610.332369	331776	2.244316
Total execution time: 0.108303s


0

# Fortran
We might like to read the gaugefield into Fortran too. While there exists code that read openqcd format into Fortran, these do not undo the checkerboarding or `flattening` of the space-time dimensions. Here we seek to have the data in the same $N_t \times N_s^3 \times N_d \times N_c^2$ format as in the `np.ndarray`.

Unlike C which uses row-major ordering, Fortran uses column-major ordering for multidimensional arrays in linear memory. We must specify this reordering when calling `reshape` by passing the argument `order='F'`.

<div class="alert alert-block alert-info"><b>Info:</b> Python natively uses neither row nor column ordering. Instead, the allocations are made directly onto the heap (which is not necessarily contiguous) rather than the stack. However, the <code>numpy</code> package is based in C and thus follows the row-major ordering scheme.</div>

This solution is based upon [this stack overflow](https://stackoverflow.com/a/49179272) answer. This solution assumes that your Python and Fortran code use the same (little) endianess.

In [41]:
# First reorder a single gaugefield into 'fortran' order
fortOrder = gfAr[0, ...].reshape(gfAr[0, ...].shape, order='F')
# we consider a single gaugefield only as Fortran allows only rank 7 arrays 
# and saving all configurations at once would be rank 8
# This could be avoided by some sort of derived type
# Now we save
fortOrder.T.tofile('Gen2_8x24_gfAr0.fort')

There is a Fortran program `readFortran.f90` supplied. This program will read `Gen2_8x24_gfAr0.fort` in and calculate whole, spatial and temporal plaquettes. We do that now

In [42]:
# The command to compile
ccmd = 'gfortran -O3 -g -fbacktrace readFortran.f90'
os.system(ccmd)

0

In [43]:
# The command to run
rcmd = './a.out'
os.system(rcmd)

 type, sumTrp, nP, ave, time (seconds)
 whole   1079332.6885531216           663552   1.6265985010264781       0.14878800511360168     
 spatial   334722.35618422477           331776   1.0088805585220895        6.2325984239578247E-002
 temporal   744610.33236885071           331776   2.2443164435307277        6.2561005353927612E-002


0

# F2PY
Here we use the <code>F2PY</code> available through <code>NumPy</code> with documentation available [here](https://numpy.org/doc/stable/f2py/).

We have written a small module in fortran, which has the essentially the same plaquette code as in `readFortran.f90`. This is in the file `fortPlaq.f90`.

In [46]:
# First we have to 'compile' the module. Luckily we already have gfortran from earlier
ccmd = 'f2py -c fortPlaq.f90 -m fortPlaq'
os.system(ccmd)

/home/ryan/bin/miniconda3/envs/lyncs/lib/python3.10/site-packages/numpy/f2py/f2py2e.py:686: VisibleDeprecationWarning: distutils has been deprecated since NumPy 1.26.Use the Meson backend instead, or generate wrapperswithout -c and use a custom build script
  builder = build_backend(
/home/ryan/bin/miniconda3/envs/lyncs/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


running build
running config_cc
INFO: unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
INFO: unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
INFO: build_src
INFO: building extension "fortPlaq" sources
INFO: f2py options: []
INFO: f2py:> /tmp/tmp0jz_s1i7/src.linux-x86_64-3.10/fortPlaqmodule.c
creating /tmp/tmp0jz_s1i7/src.linux-x86_64-3.10
Reading fortran codes...
	Reading file 'fortPlaq.f90' (format:free)
Post-processing...
	Block: fortPlaq
			Block: types
In: :fortPlaq:fortPlaq.f90:types
get_parameters: got "eval() arg 1 must be a string, bytes or code object" on 8
In: :fortPlaq:fortPlaq.f90:types
get_parameters: got "eval() arg 1 must be a string, bytes or code object" on 8
			Block: plaq
				Block: multiplymatmat
In: :fortPlaq:fortPlaq.f90:plaq:multiplymatmat
get_parameters: got "eval() arg 1 must be a string, bytes or code object" on 8
In: :fortPlaq:fortPlaq.f90:plaq:multiply

fortPlaq.f90:12:26:

   12 |         real(dc) :: r_dc = (1.0_dp, 1.0_dp)
      |                          1


INFO: compiling Fortran sources
INFO: Fortran f77 compiler: /home/ryan/bin/miniconda3/envs/lyncs/bin/gfortran -Wall -g -ffixed-form -fno-second-underscore -fPIC -O3 -funroll-loops
Fortran f90 compiler: /home/ryan/bin/miniconda3/envs/lyncs/bin/gfortran -Wall -g -fno-second-underscore -fPIC -O3 -funroll-loops
Fortran fix compiler: /home/ryan/bin/miniconda3/envs/lyncs/bin/gfortran -Wall -g -ffixed-form -fno-second-underscore -Wall -g -fno-second-underscore -fPIC -O3 -funroll-loops
INFO: compile options: '-I/tmp/tmp0jz_s1i7/src.linux-x86_64-3.10 -I/home/ryan/bin/miniconda3/envs/lyncs/lib/python3.10/site-packages/numpy/core/include -I/home/ryan/bin/miniconda3/envs/lyncs/include/python3.10 -c'
extra options: '-J/tmp/tmp0jz_s1i7/ -I/tmp/tmp0jz_s1i7/'
INFO: gfortran:f90: /tmp/tmp0jz_s1i7/src.linux-x86_64-3.10/fortPlaq-f2pywrappers2.f90
INFO: /home/ryan/bin/miniconda3/envs/lyncs/bin/gfortran -Wall -g -Wall -g -shared /tmp/tmp0jz_s1i7/tmp/tmp0jz_s1i7/src.linux-x86_64-3.10/fortPlaqmodule.o /tmp/t

0

In [47]:
# Now import the module
import fortPlaq                               
fPlaq = fortPlaq.plaq.plaquette

# separately calculate plaq
sP, nP, time = fPlaq(gfAr[0, ...], mustart=1, muend=4, nuend=4)
print(f'calculated average plaquette {sP / nP} in {time:.2} seconds')
# calculate splaq
sP, nP, time = fPlaq(gfAr[0, ...], mustart=2, muend=4, nuend=4)
print(f'calculated average spatial plaquette {sP / nP} in {time:.2} seconds')
# calculate tplaq
sP, nP, time = fPlaq(gfAr[0, ...], mustart=1, muend=1, nuend=4)
print(f'calculated average temporal plaquette {sP / nP} in {time:.2} seconds')

calculated average plaquette 1.626598501026478 in 0.11 seconds
calculated average spatial plaquette 1.0088805585220895 in 0.055 seconds
calculated average temporal plaquette 2.2443164435307277 in 0.059 seconds


### 